In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score, roc_curve

## Load data

In [3]:
df_predictors = pd.read_pickle('../data/external/df_predictors.pkl')
s_dp = df_predictors.dp.asfreq('M')
s_ret = df_predictors['Index'].pct_change().asfreq('M')

In [4]:
div_yield = s_dp.diff()[s_dp.index.year >= 1980]
returns = s_ret[s_ret.index.year >= 1980]

In [5]:
series = returns

## Class HMM

In [59]:
class HMM:
    
    def __init__(self, emission_models=(), transition_matrix=None, start_probas=None):
        '''WIP, OK'''
        self.emission_models = emission_models
        self.transition_matrix = transition_matrix
        self.start_probas = start_probas
        
        self.k = None
        
    def _check(self):
        '''WIP'''
        assert len(self.emissions) == self.transistion_matrix.shape[0] == self.transistion_matrix.shape[0] == len(self.start_probas), 'ERROR'
    
    @property
    def steady_state(self):
        '''FIX'''
        k = self.transition_matrix.shape[0]
        steady_state = np.full(k, 1/k).reshape(1, -1) @ self.transition_matrix
        return steady_state
        
    def _initialise_baum_welch(self, Y):
        '''FIX'''
        if self.start_probas is None:
            self.start_probas = self.steady_state
        
        A = np.array(self.transition_matrix)
        models = self.emission_models
        B = self._evaluate_emission_models(Y, models)
        pi = np.array(self.start_probas).reshape(1, -1)
        return A, B, pi, models
    
    def _evaluate_emission_models(self, Y, emission_models):
        '''OK'''
        B = np.concatenate([model.pdf(Y).reshape(-1, 1) for model in emission_models], axis=1)
        return B
        
    def _forward_pass(self, A, B, pi):
        '''OK'''
        # initialise forward pass with first observation
        alpha_0 = pi * B[0]
        c_0 = 1/alpha_0.sum()
        
        # save values & scaling factor
        Alpha = alpha_0*c_0
        C = [c_0]
        
        # iterate
        for b_t in B[1:]:
            # calculate
            alpha_t = (b_t * Alpha[-1] @ A).reshape(1, -1)
            c_t = 1/alpha_t.sum()
            
            # save
            Alpha = np.concatenate((Alpha, alpha_t*c_t), axis=0)
            C += [c_t]
            
        C = np.array(C).reshape(-1, 1)
        return Alpha, C
            
    def _backward_pass(self, A, B, pi, C):
        '''OK'''
        # initialise backward pass as one
        beta_T = np.ones(pi.shape)
        
        # save values & scaling factor
        Beta = beta_T*C[-1]
        
        # iterate
        for b_t, c_t in zip(B[:0:-1],C[:-1]):
            # calculate
            beta_t = (b_t * Beta[0] @ A.T).reshape(1, -1)
            
            # save
            Beta = np.concatenate((beta_t*c_t, Beta), axis=0)
            
        return Beta
    
    def _emission_odds(self, Alpha, Beta):
        '''OK'''
        total = Alpha * Beta
        Gamma = total/total.sum(axis=1).reshape(-1, 1)
        return Gamma
    
    def _transition_odds(self, A, B, Alpha, Beta):
        '''OK'''
        Alpha_block = np.kron(Alpha[:-1], np.ones(A.shape[0]))
        B_Beta_block = np.kron(np.ones(A.shape[0]), B[1:]*Beta[1:])
        total = Alpha_block * B_Beta_block * A.reshape(1, -1)
        Xi = total/total.sum(axis=1).reshape(-1, 1)
        return Xi
        
    def _do_e_step(self, Y, A, B, pi):
        '''OK'''
        Alpha, C = self._forward_pass(A, B, pi)
        Beta = self._backward_pass(A, B, pi, C)
        Gamma = self._emission_odds(Alpha, Beta)
        Xi = self._transition_odds(A, B, Alpha, Beta)
        return Gamma, Xi
    
    def _update_transition_matrix(self, Gamma, Xi):
        '''OK'''
        numerator = Xi.sum(axis=0)
        denominator = np.kron(Gamma[:-1], np.ones(Gamma.shape[1])).sum(axis=0)
        A_ = (numerator/denominator).reshape(Gamma.shape[1], Gamma.shape[1])
        return A_
    
    def _update_parameters(self, Y, emission_models, Gamma):
        '''OK'''
        models_ = []
        for model, weights in zip(emission_models, Gamma.T):
            model.fit(Y, weights)
            models_ += [model]
        return tuple(models_)
    
    def _update_initial_state(self, Gamma):
        '''OK'''
        return Gamma[0].reshape(1, -1)
    
    def _do_m_step(self, Y, models, Gamma, Xi):
        '''OK'''
        A_ = self._update_transition_matrix(Gamma, Xi)
        models_ = self._update_parameters(Y, models, Gamma)
        pi_ = self._update_initial_state(Gamma)
        return A_, models_, pi_
    
    def _score(self, Y, emission_models, Gamma):
        '''OK'''
        B = self._evaluate_emission_models(Y, emission_models)
        score = (B * Gamma).sum()    
        return score
    
    def _update_attributes(self, A_, models_, pi_, Gamma):
        '''OK'''
        self.transition_matrix = A_
        self.emission_models = models_
        self.start_probas = pi_
        self.smoothened_probabilities = Gamma
        
    
    def _estimate_baum_welch(self, Y, max_iter=100, threshold=1e-10):
        '''WIP'''
        
        #self._check()
        
        A_, B_, pi_, models_ = self._initialise_baum_welch(Y)
        iteration = 0
        scores = {iteration: self._score(Y, models_, pi_)}
        while iteration < max_iter:
            iteration += 1
            Gamma, Xi = self._do_e_step(Y, A_, B_, pi_)
            A_, models_, pi_ = self._do_m_step(Y, models_, Gamma, Xi)
            scores[iteration] = self._score(Y, models_, Gamma)
            
            if abs(scores[iteration]-scores[iteration-1]) < threshold:
                break
                
            
            
        self._update_attributes(A_, models_, pi_, Gamma)
        
        return self, scores
            
    def fit(self, Y, method='baumwelch', **kwargs):
        '''FIX'''
        if method == 'baumwelch':
            self = self._estimate_baum_welch(self, Y, **kwargs)
        return self

In [60]:
class NormalModel:
    def __init__(self, mu=0, sigma2=1):
        self.mu = mu
        self.sigma2 = sigma2
        
    def fit(self, Y, weights=None):
        '''fits the model parameters to an observation sequence, weights are optional'''
        # prepare
        Y = np.array(Y).reshape(-1, 1)
        if weights is None:
            weights = np.ones(Y.shape)
        else:
            weights = np.array(weights).reshape(-1, 1)
        
        # estimate mean
        mean = (Y*weights).sum(axis=0)/weights.sum(axis=0)
        
        # estimate variance
        errors = (Y-mean)**2
        variance = (errors*weights).sum(axis=0)/weights.sum(axis=0)
        
        # update
        self.mu = mean
        self.sigma2 = variance
        
    def pdf(self, Y):
        '''returns the likelihood of each observation in an observation sequence'''
        pdf = 1/(self.sigma2**0.5 * np.sqrt(2*np.pi)) * np.exp(-0.5*(Y-self.mu)**2/self.sigma2)
        return pdf
    
    def score(self, Y):
        '''returns the likelihood of an observation sequence'''
        score = self.pdf(Y).sum()
        return score

In [79]:
Y = np.random.randn(1000)
#Y

In [80]:
nm = NormalModel()
nm.fit(Y)
nm.score(Y)
#nm.pdf(Y)

285.51164895446664

In [81]:
test = HMM()
test.state_probas = np.array([[0, 1]])
test.transition_matrix = np.array([[0.5, 0.5],[0.2, 0.8]])
test.emission_models = (NormalModel(mu=0, sigma2=1), NormalModel(mu=1, sigma2=1))

#A,B,pi,models = test._initialise_baum_welch(Y)

#print(test._score(Y, models, Gamma))

# Alpha_0, C = test._forward_pass(A,B,pi)
# Beta_0 = test._backward_pass(A,B,pi,C)
# Gamma_0 = test._emission_odds(Alpha_0, Beta_0)
# Xi_0 = test._transition_odds(A, B, Alpha_0, Beta_0)

#Gamma, Xi = test._do_e_step(Y, A, B, pi)

#pi_0 = test._update_initial_state(Gamma)
#A_0 = test._update_transition_matrix(Gamma, Xi)
#models_0 = test._update_parameters(Y, models, Gamma)

#A_, models_, pi_ = test._do_m_step(Y, models, Gamma, Xi)

#score = test._score(Y, models_, Gamma)
test._estimate_baum_welch(Y, max_iter=1000)

(<__main__.HMM at 0x7f14897f4390>,
 {0: 239.1659209428393,
  1: 287.8983672387337,
  2: 287.3918221329287,
  3: 286.6210280250658,
  4: 286.14374240403845,
  5: 285.8821457570864,
  6: 285.73742186922607,
  7: 285.65432875236763,
  8: 285.60468030956326,
  9: 285.5739218294391,
  10: 285.55426114549624,
  11: 285.54135514813356,
  12: 285.53268892881135,
  13: 285.52675558553426,
  14: 285.5226247231616,
  15: 285.51970663624513,
  16: 285.5176188654506,
  17: 285.5161082978598,
  18: 285.5150044102626,
  19: 285.51419049558115,
  20: 285.51358555320144,
  21: 285.51313265796614,
  22: 285.51279135308914,
  23: 285.5125325930557,
  24: 285.51233533211104,
  25: 285.51218419226615,
  26: 285.5120678500401,
  27: 285.5119779081855,
  28: 285.5119080986759,
  29: 285.5118537144929,
  30: 285.51181120106423,
  31: 285.51177786016706,
  32: 285.51175163375035,
  33: 285.5117309450203,
  34: 285.5117145808685,
  35: 285.511701604364,
  36: 285.51169128925653,
  37: 285.5116830706951,
  38: 2

In [82]:
test.transition_matrix

array([[9.99992033e-01, 7.96708481e-06],
       [9.99979092e-01, 2.09082315e-05]])

In [84]:
[(m.mu,m.sigma2) for m in test.emission_models]

[(array([-0.04892599]), array([0.980498])),
 (array([-0.04887648]), array([0.98148714]))]

In [ ]:
test = HMM()
test.state_probas = np.array([[0, 1]])
test.transition_matrix = np.array([[0.5, 0.5],[0.2, 0.8]])
test.emission_models = (sp.stats.norm(loc=0, scale=1), sp.stats.norm(loc=1, scale=1))

A,B,pi = test._initialise_baum_welch(Y)

# Alpha_0, C = test._forward_pass(A,B,pi)
# Beta_0 = test._backward_pass(A,B,pi,C)
# Gamma_0 = test._emission_odds(Alpha_0, Beta_0)
# Xi_0 = test._transition_odds(A, B, Alpha_0, Beta_0)

Gamma, Xi = test._E_step(Y, A, B, pi)

test._update_initial_state(Gamma)
test._update_transition_matrix(Gamma, Xi)
#test._update_parameters(Y, Gamma)

A_, models_, pi_ = test._M_step(Y, Gamma, Xi)

In [ ]:
A_

In [ ]:
np.kron(np.ones(2), Alpha_0)

In [ ]:
np.kron(Alpha_0, np.ones(2))

In [ ]:
Gamma

In [ ]:
Y = np.array(Y).reshape(-1, 1)
weighted_Y = Y * Gamma
means = weighted_Y.sum(axis=0)/Gamma.sum(axis=0)
Y

In [ ]:
means

In [ ]:
np.kron(np.ones(Gamma.shape[1]),Y)-means

In [ ]:
Gamma

In [ ]:
np.kron(Gamma, np.ones((1,2)))